In [6]:
!conda create -n msds python=3.10.14 -y
!conda activate msds
!pip install pandas 
!conda install -c conda-forge pycaret -y
!pip install --upgrade pycaret

Channels:
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /usr/local/Caskroom/miniforge/base/envs/msds

  added / updated specs:
    - python=3.10.14


The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/osx-64::bzip2-1.0.8-h6c40b1e_6 
  ca-certificates    pkgs/main/osx-64::ca-certificates-2024.12.31-hecd8cb5_0 
  libffi             pkgs/main/osx-64::libffi-3.4.4-hecd8cb5_1 
  ncurses            pkgs/main/osx-64::ncurses-6.4-hcec6c5f_0 
  openssl            pkgs/main/osx-64::openssl-3.0.15-h46256e1_0 
  pip                pkgs/main/osx-64::pip-25.0-py310hecd8cb5_0 
  python             pkgs/main/osx-64::python-3.10.14-h5ee71fb_1 
  readline           pkgs/main/osx-64::readline-8.2-hca72f7f_0 
  setuptools         pkgs/main/osx-64::setuptools-75.8.0-py310hecd8cb5_0 
  sqlite             pkgs/main/osx-64::sqlite-3.45.3-h6c40b1e_0 
  tk                 pkgs/main/osx-64::tk-8.6.14-h4d00af3_0 
  tzd

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import os

In [2]:
file_path = input(" Input the file path:")

In [3]:
w5 = pd.read_csv(file_path)
w5.head()

,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
0,7590-VHVEG,1.0,0,0,3,29.85,29.85,0,29.850000
1,5575-GNVDE,34.0,1,1,2,56.95,1889.50,0,55.573529
2,3668-QPYBK,2.0,1,0,2,53.85,108.15,1,54.075000
3,7795-CFOCW,45.0,0,1,1,42.30,1840.75,0,40.905556
4,9237-HQITU,2.0,1,0,3,70.70,151.65,1,75.825000


In [23]:
w5.drop(columns=['customerID'], inplace=True)
w5.isna().sum()

tenure               0
PhoneService         0
Contract             0
PaymentMethod        0
MonthlyCharges       0
TotalCharges         0
Churn                0
charge_per_tenure    0
dtype: int64

# Missing values replaced with median

In [24]:
w5['TotalCharges'].fillna(w5['TotalCharges'].median(), inplace=True) # replacing missing values with median

In [25]:
w5.isna().sum() # Running isna again to check the missing values is replaced with median values for Totalcharges column.

tenure               0
PhoneService         0
Contract             0
PaymentMethod        0
MonthlyCharges       0
TotalCharges         0
Churn                0
charge_per_tenure    0
dtype: int64

# Converting categorical variables to numeric
From the below code, converted the categorical varailable to numberic values from the following columns like PhoneService, Contract, Churn, PaymentMethod


In [26]:
w5_copy = w5.copy()
w5_copy['PhoneService'] = w5_copy['PhoneService'].replace({'No':0, 'Yes':1})
w5_copy['Contract'] = w5_copy['Contract'].replace({'Month-to-month':0, 'One year':1, 'Two year':2})
w5_copy['Churn'] = w5_copy['Churn'].replace({'No':0, 'Yes':1})
w5_copy['PaymentMethod'] = w5_copy['PaymentMethod'].replace({'Electronic check':0, 'Mailed check':1, 'Bank transfer (automatic)':2, 'Credit card (automatic)':3})
w5_copy[['PhoneService','PaymentMethod', 'Contract','Churn']]

,PhoneService,PaymentMethod,Contract,Churn
0,0,3,0,0
1,1,2,1,0
2,1,2,0,1
3,0,1,1,0
4,1,3,0,1
...,...,...,...,...
7038,1,2,1,0
7039,1,0,1,0
7040,0,3,0,0
7041,1,2,0,1


In [5]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.


In [39]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [40]:
auto_ml = setup(w5_copy, target = 'Churn')

,Description,Value
0,Session id,2314
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 8)"
4,Transformed data shape,"(7043, 8)"
5,Transformed train set shape,"(4930, 8)"
6,Transformed test set shape,"(2113, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


In [41]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7970,0.8362,0.5229,0.6456,0.5770,0.4455,0.4502,0.0600
lda,Linear Discriminant Analysis,0.7931,0.8237,0.5099,0.6391,0.5668,0.4331,0.4382,0.7600
ridge,Ridge Classifier,0.7929,0.8237,0.4587,0.6597,0.5404,0.4122,0.4240,0.0160
lightgbm,Light Gradient Boosting Machine,0.7832,0.8266,0.5007,0.6131,0.5504,0.4096,0.4138,0.7790
knn,K Neighbors Classifier,0.7627,0.7502,0.4357,0.5696,0.4928,0.3417,0.3474,0.0250
qda,Quadratic Discriminant Analysis,0.7507,0.8213,0.7355,0.5218,0.6103,0.4348,0.4488,0.0130
nb,Naive Bayes,0.7442,0.8181,0.7202,0.5131,0.5990,0.4190,0.4322,0.0140
dummy,Dummy Classifier,0.7347,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0160
svm,SVM - Linear Kernel,0.7140,0.7007,0.4273,0.5622,0.4373,0.2736,0.2976,0.0160


In [42]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2314, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
w5_copy.iloc[-5:-1].shape

(4, 8)

In [44]:
predict_model(best_model, w5_copy.iloc[-5:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,Churn,prediction_label,prediction_score
7038,24.0,1,1,2,84.800003,1990.500000,82.937500,0,0,0.7619
7039,72.0,1,1,0,103.199997,7362.899902,102.262497,0,0,0.9150
7040,11.0,0,0,3,29.600000,346.450012,31.495455,0,0,0.5709
7041,4.0,1,0,2,74.400002,306.600006,76.650002,1,1,0.5791


In [45]:
save_model(best_model, 'LDA')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges',
                                              'charge_per_tenure'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('c...
                                                         

In [46]:
import pickle

with open('LDA_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [47]:
with open('LDA_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [48]:
new_data = w5_copy.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1], dtype=int8)

In [49]:
loaded_lda = load_model('LDA')

Transformation Pipeline and Model Successfully Loaded


In [50]:
predict_model(loaded_lda, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,prediction_label,prediction_score
7041,4.0,1,0,2,74.400002,306.600006,76.650002,1,0.5791


In [64]:
from IPython.display import Code
Code('churn_new_data.py')

import pandas as pd 
from pycaret.classification import predict_model, load_model


def load_data (filepath):
    w5 = pd.read_csv(filepath)
    return w5

def make_churn_predictions(w5):
    model = load_model('LDA')
    predictions = predict_model(model, data=w5)

    print(predictions.columns)

    if 'prediction_label' in predictions.columns:
        predictions.rename(columns={'prediction_label': 'Churn_prediction'}, inplace=True)
        
        return predictions['Churn_prediction']
    else:
        raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")
    

if __name__ == "__main__":
    w5 = load_data('/Users/vinodreddy/Downloads/prepared_churn_data.csv')
    predictions = make_churn_predictions(w5)
    print('predictions:')
    print(predictions)

In [65]:
%run churn_new_data.py

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7961,0.8355,0.5147,0.6452,0.5726,0.4410,0.4458


Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'charge_per_tenure', 'Churn',
       'prediction_label', 'prediction_score'],
      dtype='object')
predictions:
0       1
1       0
2       0
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn_prediction, Length: 7043, dtype: int64


In [ ]:
using the pycaret library import the setup, compare_models, predict_model, save_model and load_model for checking datatypes of the input data and also using comparing the best model for the predict model to the score probability and predicated label.using the save model saved the LDA
    
Using the pickle save the object and the loaded the model to new data.

Printed the code using Ipython library to print the code and run the code for automation and saved the code to github